<a href="https://colab.research.google.com/github/KirPim/DS_school_2020_home_work/blob/main/HW5_lesson_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

In [2]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)


In [3]:
Base = declarative_base()

class Person(Base):
    __tablename__= 'person'
    id = Column(Integer, primary_key=True)
    name = Column(String(250), nullable=False)
    
class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)
    country = Column(String(250))
    state = Column(String(250))
    street_name = Column(String(250))
    street_number = Column(String(250))
    post_code = Column(String(250))
    person_id = Column(Integer, ForeignKey('person.id'))
    person = relationship(Person)

engine = create_engine('sqlite:///sqlalchemy_example.db')

connection = engine.connect()

Base.metadata.create_all(engine)

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

In [5]:
DBSession = sessionmaker(bind=engine)
session = DBSession()

new_person = Person(name='Nawin')
session.add(new_person)
session.commit()

new_address = Address(post_code='48103',country = 'USA', state = 'WA', street_name ='West Huron Street',street_number ='1', person=new_person)
session.add(new_address)
session.commit()

In [6]:
engine.table_names()

['address', 'person']

Добавим несколько людей с разными адресами

In [7]:
from sqlalchemy import exists
def record_exists(session, name):
    return session.query(exists().where(Person.name == name)).scalar()

In [8]:
names = ['Moshe', 'Lisa', 'Nika']
adress_info = [('141700', 'Russia', 'Moscow', 'Pervomay Street', '42'), #Moshe adress
               ('141700', 'Russia', 'Moscow', 'Pervomay Street', '42'), #Lisa adress
               ('655689', 'USA', 'CA', 'Green Street', '104')] #Nika adress

In [11]:
for i, name, adress in zip(range(len(names)), names, adress_info):
    if not record_exists(session, name):
        new_person = Person(name=name)
        new_address = Address(country=adress[1],state=adress[2], street_number=adress[4], post_code=adress[0],street_name =adress[3], person=new_person)
        session.add(new_person)
        session.add(new_address)
    else:
        print("Already exists, skipping...")

session.commit()

Проверим теперь таблицу

In [12]:
query = select([Person.id, Person.name])
display_results(query)

,id,name
0,1,Nawin
1,2,Moshe
2,3,Lisa
3,4,Nika


In [13]:
query = select([Address])
display_results(query)

,id,country,state,street_name,street_number,post_code,person_id
0,1,USA,WA,West Huron Street,1,48103,1
1,2,Russia,Moscow,Pervomay Street,42,141700,2
2,3,Russia,Moscow,Pervomay Street,42,141700,3
3,4,USA,CA,Green Street,104,655689,4


Другой способ добавления с использованием **add_all()**

In [14]:
session.add_all([Person(name='Max'),
                 Person(name='Dan'),
                 Person(name='Alex')])
session.commit()

In [15]:
query = select([Person.id, Person.name])
display_results(query)

,id,name
0,1,Nawin
1,2,Moshe
2,3,Lisa
3,4,Nika
4,5,Max
5,6,Dan
6,7,Alex


In [16]:
query = select([Address])
display_results(query)

,id,country,state,street_name,street_number,post_code,person_id
0,1,USA,WA,West Huron Street,1,48103,1
1,2,Russia,Moscow,Pervomay Street,42,141700,2
2,3,Russia,Moscow,Pervomay Street,42,141700,3
3,4,USA,CA,Green Street,104,655689,4


# Домашнее задание

###Подумайте, как добавить к произвольной таблице с известным первичным ключом константный столбец, содержащий число строк в ней. Если вы знаете, что такое оконные функции, то придумайте решение без них.

In [26]:
#Напишем запрос на примере таблицы person

query1 = '''
select *
 from person as t1
 cross join (
             select count(*) as rows_total
              from person       
             ) as t2

'''

pd.read_sql(query1, connection)

,id,name,rows_total
0,1,Nawin,7
1,2,Moshe,7
2,3,Lisa,7
3,4,Nika,7
4,5,Max,7
5,6,Dan,7
6,7,Alex,7


In [27]:
#Еще один вариант реализации подзапросом в качестве поля, без join и оконных функций

query2 = '''
select *
      ,(select count(*) from person) as rows_total
 from person

'''

pd.read_sql(query2, connection)

,id,name,rows_total
0,1,Nawin,7
1,2,Moshe,7
2,3,Lisa,7
3,4,Nika,7
4,5,Max,7
5,6,Dan,7
6,7,Alex,7


ДОП ЗАДАНИЕ**

Если человек живет в Москве, то вывести его имя, улицу и почтовый код

In [28]:
query = '''
select t1.id
      ,case when t2.state = 'Moscow' then t1.name else NULL end as name
      ,t2.street_name
      ,t2.post_code
 from person as t1
 left join address as t2
  on t1.id = t2.person_id
   and t2.state = 'Moscow'
'''
pd.read_sql(query, connection)

,id,name,street_name,post_code
0,1,None,None,None
1,2,Moshe,Pervomay Street,141700
2,3,Lisa,Pervomay Street,141700
3,4,None,None,None
4,5,None,None,None
5,6,None,None,None
6,7,None,None,None
